In [ ]:
# | default_exp _cli_code_generator

In [ ]:
# | export

from typing import *
import os

import typer

from fastkafka._components.logger import get_logger
from fastkafka._code_generator.app_description_validator import validate_app_description
from fastkafka._code_generator.plan_generator import generate_plan
from fastkafka._code_generator.app_generator import generate_app
from fastkafka._code_generator.test_generator import generate_test

In [ ]:
from typer.testing import CliRunner
import pytest
from unittest.mock import patch

from fastkafka._components.logger import suppress_timestamps

In [ ]:
# | export

logger = get_logger(__name__)

In [ ]:
suppress_timestamps()
logger = get_logger(__name__, level=20)
logger.info("ok")

[INFO] __main__: ok


In [ ]:
runner = CliRunner()

In [ ]:
# | export

OPENAI_KEY_EMPTY_ERROR = "Error: OPENAI_API_KEY cannot be empty. Please set a valid OpenAI API key in OPENAI_API_KEY environment variable and try again.\nYou can generate API keys in the OpenAI web interface. See https://platform.openai.com/account/api-keys for details."
OPENAI_KEY_NOT_SET_ERROR = "Error: OPENAI_API_KEY not found in environment variables. Set a valid OpenAI API key in OPENAI_API_KEY environment variable and try again. You can generate API keys in the OpenAI web interface. See https://platform.openai.com/account/api-keys for details."


def _ensure_openai_api_key_set() -> None:
    """Ensure the 'OPENAI_API_KEY' environment variable is set and is not empty.

    Raises:
        KeyError: If the 'OPENAI_API_KEY' environment variable is not found.
        ValueError: If the 'OPENAI_API_KEY' environment variable is found but its value is empty.
    """
    try:
        openai_api_key = os.environ["OPENAI_API_KEY"]
        if openai_api_key == "":
            raise ValueError(OPENAI_KEY_EMPTY_ERROR)
    except KeyError:
        raise KeyError(OPENAI_KEY_NOT_SET_ERROR)

In [ ]:
with patch.dict(os.environ, {"OPENAI_API_KEY": ""}):
    with pytest.raises(ValueError) as e:
        _ensure_openai_api_key_set()

print(e.value)
assert str(e.value) == OPENAI_KEY_EMPTY_ERROR

Error: OPENAI_API_KEY cannot be empty. Please set a valid OpenAI API key in OPENAI_API_KEY environment variable and try again.
You can generate API keys in the OpenAI web interface. See https://platform.openai.com/account/api-keys for details.


In [ ]:
with patch.dict(os.environ, {}, clear=True):
    with pytest.raises(KeyError) as e:
        _ensure_openai_api_key_set()
        
print(e.value)
assert str(e.value) == f"'{OPENAI_KEY_NOT_SET_ERROR}'"

'Error: OPENAI_API_KEY not found in environment variables. Set a valid OpenAI API key in OPENAI_API_KEY environment variable and try again. You can generate API keys in the OpenAI web interface. See https://platform.openai.com/account/api-keys for details.'


In [ ]:
with patch.dict(os.environ, {"OPENAI_API_KEY": "INVALID_KEY"}):
    _ensure_openai_api_key_set()

In [ ]:
# | export

_code_generator_app = typer.Typer(
    short_help="Commands for accelerating FastKafka app creation using advanced AI technology",
     help="""Commands for accelerating FastKafka app creation using advanced AI technology.

These commands uses OpenAI's GPT-3.5 API for generating FastKafka code. To access this feature, kindly sign up if you haven't already and create an API key with OpenAI. If you're unsure about creating a new OpenAI API key, check this link for guidance: https://help.openai.com/en/articles/4936850-where-do-i-find-my-secret-api-key.

Once you have the key, please set it in the OPENAI_API_KEY environment variable before executing the code generation commands.

Note: Accessing OpenAI API incurs charges. However, when you sign up for the first time, you usually get free credits that are more than enough to generate multiple FastKafka applications. For further information on pricing and free credicts, check this link: https://openai.com/pricing
    """,
)

In [ ]:
# | export


@_code_generator_app.command(
    "generate",
    help="Generate a new FastKafka app(s) effortlessly with advanced AI assistance",
)
def generate_fastkafka_app(
    description: str = typer.Argument(
        ...,
        help="""Summarize your FastKafka app in a few sentences!


\nInclude details about message classes, FastKafka app configuration (e.g., kafka_brokers), consumer and producer functions, and specify the business logic to be implemented. 


\nThe simpler and more specific the app description is, the better the generated app will be. Please refer to the below example for inspiration:


\nCreate a FastKafka application that consumes messages from the "store_product" topic. These messages should have three attributes: "product_name," "currency," and "price". While consuming, the app needs to produce a message to the "change_currency" topic. The function responsible for producing should take a "store_product" object as input and return the same object. Additionally, this function should check if the currency in the input "store_product" is "HRK." If it is, then the currency should be changed to "EUR," and the price should be divided by 7.5. Remember, the app should use a "localhost" broker.


\n"""
    )
) -> None:
    """Generate a new FastKafka app(s) effortlessly with advanced AI assistance"""
    try:
        _ensure_openai_api_key_set()
        validated_description, description_token = validate_app_description(description)
        plan = generate_plan(validated_description)
        code = generate_app(plan)
        test = generate_test(code)
        
        typer.secho(f" ▶ Total tokens usage: {description_token}", fg=typer.colors.CYAN)
        typer.secho("✨  All files were successfully generated.!", fg=typer.colors.CYAN)
    
    except (ValueError, KeyError) as e:
        typer.secho(e, err=True, fg=typer.colors.RED)
        raise typer.Exit(code=1)
    except Exception as e:
        typer.secho(f"Unexpected internal error: {e}", err=True, fg=typer.colors.RED)
        raise typer.Exit(code=1)

In [ ]:
# | notest

! nbdev_export

In [ ]:
result = runner.invoke(_code_generator_app, ["generate", "--help"])

                                                                                                                   
 Usage: generate [OPTIONS] DESCRIPTION                                                                             
                                                                                                                   

Generate a new FastKafka app(s) effortlessly with advanced AI assistance

╭─ Arguments ─────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ *    description      TEXT  Summarize your FastKafka app in a few sentences!                                    │
│                                                                                                                 │
│                             Include details about message classes, FastKafka app configuration (e.g.,           │
│                             kafka_brokers), consumer and producer functions, and specify the business logic to  │
│                             be implemented.                                                                     │
│                                                                                                                 │
│                             The simpler and more specific the app description is, the better the generated app  │
│                             will be. Please refer to the below example for inspiration:                         │
│                                                                                                                 │
│                             Create a FastKafka application that consumes messages from the "store_product"      │
│                             topic. These messages should have three attributes: "product_name," "currency," and │
│                             "price". While consuming, the app needs to produce a message to the                 │
│                             "change_currency" topic. The function responsible for producing should take a       │
│                             "store_product" object as input and return the same object. Additionally, this      │
│                             function should check if the currency in the input "store_product" is "HRK." If it  │
│                             is, then the currency should be changed to "EUR," and the price should be divided   │
│                             by 7.5. Remember, the app should use a "localhost" broker.                          │
│                             [default: None]                                                                     │
│                             [required]                                                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Options ───────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ --install-completion          Install completion for the current shell.                                         │
│ --show-completion             Show completion for the current shell, to copy it or customize the installation.  │
│ --help                        Show this message and exit.                                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
# result = runner.invoke(_code_generator_app, ["Sample FastKafka application description"])
# assert result.exit_code == 0, result.exit_code